In [1]:
import os
import random
import numpy as np

import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from neural_predictor import regressor
from input_preprocessing import preprocess_nasbench_nlp
from search_spaces import load_nasbench_nlp

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

### Test on New Benchmark Datasets (NLP)

In [3]:
search_set, env = load_nasbench_nlp()

In [4]:
LOOPS = 60
MAX_SAMPLES = 500
MAX_TIME_BUDGET = 8e5

### Oracle

In [5]:
val_loss, test_loss = [], []
    
env.reset()
for s in search_set:
    env.simulated_train(s, 50)
    if 'OK' == env.get_model_status(s):
        stats = env.get_model_stats(s, 49)
        val_loss.append(stats['val_loss'])
        test_loss.append(stats['test_loss'])

In [6]:
print(np.min(val_loss), np.std(val_loss))
print(np.min(test_loss), np.std(test_loss))

4.41530952065678 0.5854086570687619
4.368057399120145 0.5847845925439686


In [7]:
oracle_loss = {'val_loss': np.min(val_loss), 'test_loss': np.min(test_loss)}
oracle_std = {'val_std': np.std(val_loss), 'test_std': np.std(test_loss)}

In [8]:
np.save('outputs/nlp_oracle_loss.npy', oracle_loss)
np.save('outputs/nlp_oracle_std.npy', oracle_std)

### Random Search

In [9]:
env.reset()

random_val_avg, random_test_avg = [], []
random_val_std, random_test_std = [], []

val_loss, test_loss = [], []
for _ in tqdm(range(MAX_SAMPLES)):
    loop_val, loop_test = [], []
    for _ in range(LOOPS):
        idx = np.random.choice(len(search_set), 1, replace=False)[0]
        s = search_set[idx]

        env.simulated_train(s, 50)
        if 'OK' == env.get_model_status(s):
            stats = env.get_model_stats(s, 49)
            loop_val.append(stats['val_loss'])
            loop_test.append(stats['test_loss'])

    val_loss.append(np.mean(loop_val))
    test_loss.append(np.mean(loop_test))
    
    random_val_avg.append(np.min(val_loss))
    random_val_std.append(np.std(loop_val))
    
    random_test_avg.append(np.min(test_loss))
    random_test_std.append(np.std(loop_test))

  0%|          | 0/500 [00:00<?, ?it/s]

In [10]:
np.save('outputs/nlp_random_val_avg_by_samples.npy', random_val_avg)
np.save('outputs/nlp_random_val_std_by_samples.npy', random_val_std)
np.save('outputs/nlp_random_test_avg_by_samples.npy', random_test_avg)
np.save('outputs/nlp_random_test_std_by_samples.npy', random_test_std)

### Neural Predictor

In [17]:
N = 172
def get_N_samples(N):
    env.reset()
    
    models = []
    stats = []

    for _ in range(N):
        while True:
            idx = np.random.choice(len(search_set), 1, replace=False)[0]
            model = search_set[idx]
            
            if model not in models:
                env.simulated_train(model, 50)
                
                if 'OK' == env.get_model_status(model):   
                    models.append(model)
                    stats.append(env.get_model_stats(model, 49))
                    break
                
    return preprocess_nasbench_nlp(models, stats)

In [18]:
# training part
train_data = get_N_samples(N)
reg = regressor([train_data['X'], train_data['norm_A'], train_data['norm_AT']], train_data['val_loss'], mode='loss')

In [19]:
# testing part
np_val_avg, np_test_avg = [], []
np_val_std, np_test_std = [], []

val_acc, test_acc = [], []
for k in tqdm(range(MAX_SAMPLES - N)):
    K = k + 1

    loop_val, loop_test = [], []
    for _ in range(LOOPS):
        test_models = get_N_samples(N+K)
        pred_acc = reg.predict([test_models['X'], test_models['norm_A'], test_models['norm_AT']]).ravel()
        
        topk_idx = tf.math.top_k(-pred_acc, k=K).indices.numpy()
        selected_val = test_models['val_loss'][topk_idx]
        selected_test = test_models['test_loss'][topk_idx]
        
        best_val_idx = np.argmin(selected_val)
        
        loop_val.append(selected_val[best_val_idx])
        loop_test.append(selected_test[best_val_idx])

    val_acc.append(np.mean(loop_val))
    test_acc.append(np.mean(loop_test))
    
    np_val_avg.append(np.min(val_acc))
    np_val_std.append(np.std(loop_val))
    
    np_test_avg.append(np.min(test_acc))
    np_test_std.append(np.std(loop_test))

  0%|          | 0/328 [00:00<?, ?it/s]

In [31]:
np.save('outputs/nlp_np_val_avg_by_samples.npy', np_val_avg)
np.save('outputs/nlp_np_val_std_by_samples.npy', np_val_std)
np.save('outputs/nlp_np_test_avg_by_samples.npy', np_test_avg)
np.save('outputs/nlp_np_test_std_by_samples.npy', np_test_std)